In [1]:
import pandas as pd
from konlpy.tag import Hannanum
from pykospacing import Spacing
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
data = pd.read_excel('../../../../../data/filtered_news.xlsx')
# data = data[:100]

spacing = Spacing()
processed_features = []
for feature in data['feature']:
    processed_feature = spacing(feature)
    processed_features.append(processed_feature)
data['feature'] = processed_features

In [5]:
# Hannanum 형태소 분석기로 토큰화 (for문 사용)
hannanum = Hannanum()
tokens = []
for feature in data['feature']:
    tokenized = hannanum.nouns(feature)
    tokens.append(tokenized)
data['tokens'] = tokens

In [ ]:
# 불용어 처리 (for문 사용)
with open('../../../../../data/stopwords-ko.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

filtered_tokens = []
for token_list in data['tokens']:
    filtered = [word for word in token_list if word not in stopwords]
    filtered_tokens.append(filtered)
data['tokens'] = filtered_tokens

In [7]:
data['tokens_str'] = data['tokens'].apply(lambda x: ' '.join(x))               # 리스트를 문자열로 변환

vectorizer = CountVectorizer()                                                 # CountVectorizer를 사용하여 BoW 모델 생성
X = vectorizer.fit_transform(data['tokens_str'])

bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out()) # BoW 결과를 데이터프레임으로 변환

df_bow = pd.concat([data, bow_df], axis=1)                                     # 원래 데이터프레임과 BoW 데이터프레임을 병합